In [ ]:
import pandas as pd
from pymongo import MongoClient
client = MongoClient()
db = client['cdb']
col = db['arr']

In [ ]:
def get_table(col, fields, id_field):
    fields_keys = fields.keys()
    fields_keys2 = fields_keys+[id_field]
    query_params = {k:1 for k in fields_keys2}
    docs = col.find({},query_params)
    
    data = []
    idx = []
    acc = 0
    for doc in docs:
        data.append([doc[k] for k in fields_keys])
        idx.append(doc[id_field])
        acc += 1
    
    return pd.DataFrame(data,columns = fields.values(),index=idx)

In [ ]:
personal_data_key_dict = {
    u'Age':'age',
    u'ChNum':'children_number',
    u'Edu':'has_high_education',
    u'Mar':'is_maried',
    u'Name':'name',
    u'SocialStat':'social_status',
    u'State':'state',
    u'WorkExp':'work_experience',
    u'ZipCode':'zip',
    u'Contact':'can_be_contacted'
    }

df = get_table(col, personal_data_key_dict, 'X_id')
df.to_csv('data/01_personal_data.csv')

In [ ]:
bank_comm_data_key_dict = {
    u'ATMUser':'used_atm_last_month',
    u'CallCenter':'used_call_center_last_month',
    u'LastContactData':'time_from_last_contact_date',
    u'LifeChat':'used_life_chat_last_month',
    u'PreferredContactChannel':'prefered_channel',
    u'Website':'used_website_last_month',
    u'has_fb_acc':'has_fb_account'
    }

df = get_table(col, bank_comm_data_key_dict, 'X_id')
df.to_csv('data/02_bank_comm_data.csv')

In [ ]:
bank_fin_data_key_dict = {
    u'BalanceDeposit':'deposit_balance',
    u'BalanceLoan':'loan_balance',
    u'CC':'has_credit_card',
    u'CCBalance':'credit_card_balance',
    u'Checking':'has_checking_account',
    u'CheckingBalance':'checking_account_balance',
    u'DepositCP':'deposit_products_profitability',
    u'Home_eq':'has_home_equity_loan',
    u'Home_eqBalance':'home_equity_loan_balance',
    u'LoansCP':'loan_products_profitability',
    u'MM':'has_money_market_product',
    u'MMBalance':'money_market_balance',
    u'Mortgage':'has_mortgage',
    u'MortgageBalance':'mortgage_balance',
    u'NIIDeposit':'deposit_products_nii',
    u'NIILoan':'loan_products_nii',
    u'REStat':'real_estate_status',
    u'StLoan':'has_student_loan',
    u'StLoanBalance':'student_loan_balance',
    u'TotalCP':'customer_profitability',
    u'Segment':'segment'
}

df = get_table(col, bank_fin_data_key_dict, 'X_id')
df.to_csv('data/03_bank_fin_data.csv')


In [ ]:
bank_nbo_model_data_key_dict = {
    u'nCC':'credit_card_nbo',
    u'nHome_eq':'home_equity_loan_nbo',
    u'nMortgage':'mortgage_nbo',
    u'nStLoan':'student_loan_nbo',
    u'nChecking':'checking_account_nbo',
    u'nMM':'money_market_nbo'
}

df = get_table(col, bank_nbo_model_data_key_dict, 'X_id')
df.to_csv('data/04_bank_nbo_model_data.csv')


In [ ]:
bank_cltv_model_data_key_dict = {
    u'fp1y_prod':'cltv'
}

df = get_table(col, bank_cltv_model_data_key_dict, 'X_id')
df.to_csv('data/05_bank_cltv_model_data.csv')


In [ ]:
bank_external_data_key_dict = {
    u'CreditScore':'credit_score',
    u'InvestableAssetIndicator':'investable_asset_indicator'
}

df = get_table(col, bank_external_data_key_dict, 'X_id')
df.to_csv('data/06_bank_external_data.csv')


In [ ]:
"""
bank_transaction_data_key_dict = {
    u'geo.us':'geo_part_of_transactions_from_us',
    u'geo.eu':'geo_part_of_transactions_from_eu',
    u'geo.asia':'geo_part_of_transactions_from_asia',
    u'geo.africa':'geo_part_of_transactions_from_africa',
    u'curr.usd':'curr_part_of_transactions_in_usd',
    u'curr.eu':'curr_part_of_transactions_in_eu',
    u'curr.other':'curr_part_of_transactions_in_other_currency',
    u'type.exch':'type_part_of_currency_exchange_transactions',
    u'type.transf':'type_part_of_transfer_transactions',
    u'type.buy':'type_part_of_shopping_transactions',
    u'type.other':'type_part_of_other_transactions',
    u'method.atm':'method_part_of_transactions_done_with_atm',
    u'method.card':'method_part_of_transactions_done_with_card_offline',
    u'method.ecom':'method_part_of_transactions_done_to_ecommerce_site',
    u'method.webbank':'method_part_of_transactions_done_with_webbank',
    u'method.mobile':'method_part_of_transactions_done_with_mobile_bank',
    u'dest.mixed':'dest_part_of_transactions_done_to_mixed_dest',
    u'dest.tech':'dest_part_of_transactions_done_to_electronics',
    u'dest.restaurants':'dest_part_of_transactions_done_in_restraunts',
    u'dest.grocery':'dest_part_of_transactions_done_in_grocery',
    u'dest.serv':'dest_part_of_transactions_done_for_service',
    u'dest.travel':'dest_part_of_transactions_done_for_travel_service',
    u'dest.other':'dest_part_of_transactions_done_for_other_destinations'
}
"""

In [ ]:
df = pd.read_csv('./LM_dataset.csv', ' ')

In [ ]:
df_ref = pd.read_csv('data/03_bank_fin_data.csv')
valid_ids = (df_ref['has_credit_card'] + df_ref['has_checking_account'])>0.5
df_ref = pd.read_csv('data/01_personal_data.csv')
age_ref = df_ref[valid_ids]['age']
age_ref = age_ref.to_dict()

In [ ]:
idx = []
for age in df['age']:
    for (id_,age2) in age_ref.iteritems():
        if abs(age2 - age) < 5:
            idx.append(id_)
            age_ref[id_] = 200
            break
    if not len(idx)%1000:
        age_ref = {k:v for k,v in age_ref.iteritems() if v<200}
        print len(idx)

In [ ]:
transaction_keys = [u'geo.us', u'geo.eu', u'geo.asia', u'geo.africa', u'curr.usd',
       u'curr.eu', u'curr.other', u'type.exch', u'type.transf', u'type.buy',
       u'type.other', u'method.atm', u'method.card', u'method.ecom',
       u'method.webbank', u'method.mobile', u'dest.mixed', u'dest.tech',
       u'dest.restaurants', u'dest.grocery', u'dest.serv', u'dest.travel',
       u'dest.other']

behavior_keys = [u'commchan.mobileApp', u'commchan.web', u'commchan.ATM',
       u'commchan.phone', u'dur.lt1m', u'dur.lt5m', u'dur.mt5m', u'sect.card',
       u'sect.account', u'sect.loan', u'sect.offers', u'act.yes',
       u'adresp.trav', u'adresp.clothes', u'adresp.rest', u'adresp.car']

model_output_keys = [u'cluster_bh', u'cluster_tr']

In [ ]:
df.index = idx
df[transaction_keys]


In [ ]:
df[transaction_keys].to_csv('data/07_bank_transaction_data.csv')
df[behavior_keys].to_csv('data/08_bank_behavior_data.csv')
df[model_output_keys].to_csv('data/09_cluster_data.csv')


In [ ]:
import hadoopy

#hadoopy.mkdir('/user/smartanalytics/customer_data')
p2 = '/user/smartanalytics/customer_data/'
p = '/data/'
hadoopy.put(p+'01_personal_data.csv',p2+'01_personal_data.csv')
hadoopy.put(p+'02_bank_comm_data.csv',p2+'02_bank_comm_data.csv')
hadoopy.put(p+'03_bank_fin_data.csv',p2+'03_bank_fin_data.csv')
hadoopy.put(p+'04_bank_nbo_model_data.csv',p2+'04_bank_nbo_model_data.csv')
hadoopy.put(p+'05_bank_cltv_model_data.csv',p2+'05_bank_cltv_model_data.csv')
hadoopy.put(p+'06_bank_external_data.csv',p2+'06_bank_external_data.csv')
hadoopy.put(p+'07_bank_transaction_data.csv',p2+'07_bank_transaction_data.csv')
hadoopy.put(p+'08_bank_behavior_data.csv',p2+'08_bank_behavior_data.csv')
hadoopy.put(p+'09_cluster_data.csv',p2+'09_cluster_data.csv')

In [ ]:
hadoopy.ls('/user/smartanalytics/customer_data/')